<a href="https://colab.research.google.com/github/gcourtade/GeqShift/blob/main/GeqShift.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##GeqShift

Easy to use carbohydrate <sup>13</sup>C NMR chemical shift prediction using GeqShift: an E(3) equivariant graph neural network.

The original GeqShift code is available at https://github.com/mariabankestad/GeqShift.

The dataset of 1H and 13C NMR chemical shifts are available at https://github.com/mariabankestad/GeqShift.

Please read and cite the GeqShit paper:
[Bånkestad M., Dorst K. M., Widmalm G., Rönnols J. Carbohydrate NMR chemical shift prediction by GeqShift employing E(3) equivariant graph neural networks
*RSC Advances*, 2024](https://doi.org/10.1039/D4RA03428G)

##### Disclaimer
I made this Google Colab notebook for my own use and have no connection with the authors of the GeqShift paper. This notebook was is heavily inspired by and uses code from the [ColabFold](https://colab.research.google.com/github/sokrypton/ColabFold/blob/main/AlphaFold2.ipynb#scrollTo=mbaIO9pWjaN0) notebook. The model was trained using 100 conformations of the carbohydrates in the training set and 20 epochs. I cannot guarantee the correctness of the results generated using this code.

--[Gaston Courtade](https://folk.ntnu.no/courtade), 2024-08-24

In [5]:
#@title Input carbohydrate SMILES, then hit `Runtime` -> `Run all`
import os


query_smiles = 'C([C@@H]1[C@H]([C@@H]([C@H](C(O1)O)O)O)O)O' #@param {type:"string"}
#@markdown  - Tip: Use [SMILES generator/checker](https://www.cheminfo.org/flavor/malaria/Utilities/SMILES_generator___checker/index.html) to edit SMILES.
jobname = 'beta-glucose' #@param {type:"string"}
# number of models to use
num_conformations = 1 #@param {type: "integer"}
#@markdown - Specify how many conformations should be generated in the ensemble for chemical shift prediction
#@markdown - Tip: Best results are expected with around 100 conformations

if os.path.exists("predict"):
  !rm -r /content/predict

In [6]:
#@title Install dependencies

import torch

torch_url = "https://pytorch-geometric.com/whl/torch-{}.html".format(torch.__version__).replace('+', '%2B')
!pip install torch-cluster torch-geometric torch-scatter -f $torch_url
!pip install e3nn
!pip install rdkit
!pip install mdanalysis
!pip install py3Dmol

if not os.path.exists("GeqShift"):
  !git clone https://github.com/gcourtade/GeqShift.git

checkpoint_file = "_20240823_checkpoint_epoch-20_nbr-confs-100.pkl"
if not os.path.exists(checkpoint_file):
  checkpoint_url = "https://folk.ntnu.no/courtade/GeqShift_models/" + checkpoint_file
  !curl -O $checkpoint_url

!mv GeqShift/predict_gpu.py .

Looking in links: https://pytorch-geometric.com/whl/torch-2.4.0%2Bcu121.html


In [9]:
#@title Run prediction
!python predict_gpu.py --smiles_list "$query_smiles" --mol_name "$jobname" --checkpoint_path $checkpoint_file --nbr_confs $num_conformations

/content/GeqShift/model/norms.py:164: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @torch.cuda.amp.autocast(enabled=False)
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, }
Loaded checkpoint from _20240823_checkpoint_epoch-20_nbr-confs-100.pkl
Predict chemical shifts ...
Predictions saved to predict//glucose_predictions.pkl
[12:44:04] Molecule does not have explicit Hs. Consider calling AddHs()
[12:44:04] Molecule does not have explicit Hs. Consider calling AddHs()
Coordinates saved to predict//glucose.pdb
Coordinates with shifts in Bfactor col saved to predict//glucose_shifts.pdb
Predicted chemical shifts:
13C_idx, CS
0 61.85
1 72.39
2 70.63
3 73.83
4 72.47
5 93.02


In [10]:
#@title Display 3D structure with <sup>13</sup>C chemical shifts

import py3Dmol
import glob

def parse_pdb(pdb_file):
    atoms = []
    with open(pdb_file, 'r') as file:
        for line in file:
            if line.startswith("HETATM"):
                # Extract atom name, coordinates, and B-factor
                atom_name = line[12:16].strip()
                x = float(line[30:38].strip())
                y = float(line[38:46].strip())
                z = float(line[46:54].strip())
                bfactor = float(line[60:66].strip())
                atoms.append((atom_name, x, y, z, bfactor))
    return atoms

pdb_filename = f"/content/predict/{jobname}_shifts.pdb"
pdb_file = glob.glob(pdb_filename)
atoms = parse_pdb(pdb_filename)

view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js',)
view.addModel(open(pdb_file[0],'r').read(),'pdb')

view.setStyle({'stick': {}})

for atom_name, x, y, z, bfactor in atoms:
    if bfactor != 0:
        label_content = f"{bfactor:.2f}"
        view.addLabel(label_content, {'position': {'x': x, 'y': y, 'z': z}, 'fontSize': 12})

view.zoomTo()

view.show()


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [11]:
#@title Download the results
zip_path = f"/content/{jobname}.zip"
!zip -r  $zip_path /content/predict
from google.colab import files
files.download(zip_path)

  adding: content/predict/ (stored 0%)
  adding: content/predict/glucose_conformations.pickle (deflated 0%)
  adding: content/predict/glucose_pred_data.pkl (deflated 74%)
  adding: content/predict/glucose.pdb (deflated 73%)
  adding: content/predict/glucose_shifts.pdb (deflated 65%)
  adding: content/predict/glucose_predictions.pkl (deflated 17%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

The chemical shifts are stored in the B-factor column of the _shifts.pdb file. To visualize chemical shifts on the structure, you can label the atoms by B-factor:
1. Open `JOBNAME_shifts.pdb ` on PyMOL
2. Type `label all, b` or click `L > b-factor`